In [121]:
import os
import re

import pandas

In [122]:
class NumberExtraction:

    @staticmethod
    def extract_decimals_from_text(text : str):
        # ======================================
        # 
        # Decimal extraction steps:
        # Step 0: Remove all commas between numbers
        #
        # ======================================
        
        extracted_decimals = re.findall(r'-?\d+\.?\d*', text)
        return extracted_decimals

In [123]:
def is_sympy_equal_final_answer(row):
    solved = row['majority_solved']
    solved = NumberExtraction.extract_decimals_from_text(solved)
    solved = [float(s) for s in solved]
    solved = [round(s, 2) for s in solved]
    solved = set(solved)

    majority_answer = row['majority_answer']
    majority_answer = [round(s, 2) for s in majority_answer]
    majority_answer = set(majority_answer)

    return majority_answer.issubset(solved) and len(majority_answer) > 0

In [124]:
from typing import List
from collections import Counter
from scipy.stats import entropy

def probability_distribution(values : List[any]):
    counter = Counter(values)

    for key in counter.keys():
        counter[key] = counter[key] / len(values)
    
    return counter

class ShannonEntropy:
    # TODO: What should I name this? It's basically something that looks at something, 
    # calculates the probability of it appearing and then calculates the shannon entropy.
    def base(values : List[any], **kwargs) -> float:
        values = [str(value) for value in values]
        prob = probability_distribution(values).values()
        return entropy(pk=list(prob), **kwargs)

In [125]:
majority = pandas.read_json('data/output/majority_set/sample_0.jsonl', lines=True)
majority['sympy_equals_final_answer'] = majority.apply(lambda row : is_sympy_equal_final_answer(row), axis=1)
majority.head(5)

,question_id,majority_answer,majority_answer_freq,majority_equation,majority_equation_freq,majority_solved,majority_solved_freq,sympy_equals_final_answer
0,40964,[1155],9,x/42 = 27.5/1\nx = 42 * 27.5\nx = 1155,2,{a: 1155.00000000000},8,True
1,118870,[73],10,508 = 6x + 70\n438 = 6x\nx = 73,4,{a: 73},10,True
2,190835,[4],10,x + (2x + 1) = 13\n3x + 1 = 13\n3x = 12\nx = 4,3,{a: 4},7,True
3,178255,[332],10,"Total cost = 54,150 + 12.5x \nRevenue = 25x \n...",1,{'a': 4332.0},8,False
4,223977,"[21, 26]",10,x + y = 47\nx - y = 5,6,"{a: 26, b: 21}",9,True


In [126]:
chatgpt_answers = pandas.DataFrame()
for filename in os.listdir('data/output/chatgpt_answers'):
    chatgpt_answer = pandas.read_json(f'data/output/chatgpt_answers/{filename}', lines=True)
    chatgpt_answers[filename] = chatgpt_answer['chatgpt_answer']
chatgpt_answers.head(1)

,sample_0.jsonl,sample_1.jsonl,sample_2.jsonl,sample_3.jsonl,sample_4.jsonl,sample_5.jsonl,sample_6.jsonl,sample_7.jsonl,sample_8.jsonl,sample_9.jsonl
0,[1155],[1155],[1155],[155],[1155],[1155],[1155],[1155],[1155],[1155]


In [127]:
def calculate_entropy(row):
    arr = []
    for filename in os.listdir('data/output/chatgpt_answers'):
        arr.append(row[filename])
    return ShannonEntropy.base(arr)

chatgpt_answers['entropy'] = chatgpt_answers.apply(lambda row : calculate_entropy(row), axis=1)
chatgpt_answers.head(1)

,sample_0.jsonl,sample_1.jsonl,sample_2.jsonl,sample_3.jsonl,sample_4.jsonl,sample_5.jsonl,sample_6.jsonl,sample_7.jsonl,sample_8.jsonl,sample_9.jsonl,entropy
0,[1155],[1155],[1155],[155],[1155],[1155],[1155],[1155],[1155],[1155],0.325083


In [ ]:
def generate_geq(row, column_name):
    for i in range(1, row[column_name] + 1):
        row[f'{column_name}__{i}'] = 1
    return row

In [128]:
chatgpt_answers['entropy'].value_counts().sort_index()

entropy
0.000000    417
0.325083    179
0.500402     37
0.610864     14
0.639032     70
0.673012     11
0.801819     20
0.897946      9
0.940448     45
0.943348      7
0.943348      5
0.943348      2
0.950271      3
1.029653      1
1.054920      1
1.088900      7
1.088900      7
1.168282     13
1.193550      2
1.220607      5
1.227529     28
1.279854      1
1.313834      1
1.332179      1
1.359237     15
1.359237      2
1.418484      8
1.418484      2
1.470808      1
1.470808      3
1.497866      1
1.497866      6
1.504788      4
1.504788      1
1.609438      3
1.609438      8
1.609438      1
1.643418      1
1.643418      1
1.695743      2
1.695743      3
1.695743      1
1.748067      3
1.748067      9
1.834372      5
1.834372      8
1.886697      1
1.886697      1
1.886697      1
1.973001      7
2.025326      4
2.163956      8
2.302585      3
Name: count, dtype: int64